In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

memory = ConversationBufferMemory(return_messages=True,
                                  memory_key="chat_history",
                                  human_prefix="human",
                                  ai_prefix="ai")

examples = [
    {
        "title": "Top Gun",
        "emojis": "🛩️👨‍✈️🔥"
    },
    {
        "title": "The Godfather",
        "emojis": "👨‍👨‍👦🔫🍝"
    },
    {
        "title": "The Matrix",
        "emojis": "🕶️💊🤖"
    },
    {
        "title": "The Lion King",
        "emojis": "🦁👑🌅"
    },
    {
        "title": "The Dark Knight",
        "emojis": "🦇🃏💣"
    },
]

example_prompt = ChatPromptTemplate.from_messages([("system", "{title}"),
                                                   ("system", "{emojis}")])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples, example_prompt=example_prompt)

final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a emoji movie bot. I will give you a movie title and you will respond with three emojis that represent the movie."
     ), ("system", "Here are some examples:"), few_shot_prompt,
    ("system", "Now it's your turn!"),
    MessagesPlaceholder(variable_name="chat_history"), ("human", "{title}")
])


def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


chain = RunnablePassthrough.assign(
    chat_history=load_memory) | final_prompt | llm


def invoke_chain(title):
    result = chain.invoke({"title": title})
    memory.save_context({"input": title}, {"output": result.content})
    print(result.content)

In [11]:
invoke_chain("The Dark Knight Rises")
invoke_chain("The Man of Steel")

🦇🔥💥
🦸‍♂️⚔️🔥


In [12]:
invoke_chain("What was my first input movie?")

Your first input movie was "The Dark Knight Rises."
